### Automation Script Overhaul 
For automating the parsing, transformation, and geographic file creation from .GPX<br><br>
Work taken from the script by Isias (ler_gpx.py) and workflow by Simone. Further developments by Kyle & Isais. <br><br>
Data collection must be done with Locus Map 4.x or formatted similarly to GPX files exportd by Locus Map 4 for this script to work.

### Section 1
This section takes the raw GPX files and makes them ready for QGIS and statistical analysis.

In [1]:
import gpxpy
import pandas as pd
import geopandas as gpd
import os
from os import mkdir


gpxDict = dict()

for file in os.listdir():
    if file.endswith('.gpx'):
       gpxDict[file] = 'file_'+file
gpxDict

{'20220607_Kyle.gpx': 'file_20220607_Kyle.gpx',
 '20220610_Kyle.gpx': 'file_20220610_Kyle.gpx'}

In [2]:
# merged = pd.DataFrame()

for i in gpxDict:
    
    # Open and read in the .gpx to a dataframe / Abra e leia no .gpx para um dataframe
    gpxCurrent = i
    gpxCurrent = open(gpxCurrent)
    gpxCurrent = gpxpy.parse(gpxCurrent)
    gpxCurrent = gpxCurrent.to_xml()
    df = pd.read_xml(gpxCurrent)

    # Ask for observer, group, climate conditions / Pergunte por observador, grupo, condições climáticas
    observer = input('Input for '+i+': Observer/Observador? ')
    group = input('Input for '+i+': Group/Grupo? (if both, mark 0) ') # or leave blank?
    weather = input('Input for '+i+': Weather conditions/Condição do clima? ')
    
    # Remove unecessary columns / Remova colunas desnecessárias
    df.pop('desc')
    df.pop('hdop')
    df.pop('time')
    df = df.drop(index=0)

    # Reorganize columns / Reorganizar colunas
    shiftPos = df.pop('name')
    df.insert(0, 'name', shiftPos)

    # Insert user input columns / Inserir colunas de entrada do usuário
    df.insert(loc=1, column='observer', value=observer, allow_duplicates=True)
    df.insert(loc=1, column='group', value=group, allow_duplicates=True)
    df.insert(loc=1, column='weather', value=weather, allow_duplicates=True)

    # Split 'name' into date, time, and observations / Dividir 'nome' em data, hora e observações
    date = df['name'].str[:10]
    df.insert(loc=0, column='date', value=date, allow_duplicates=True)

    time = df['name'].str[11:19]
    df.insert(loc=1, column='time', value=time, allow_duplicates=True)

    obs = df['name'].str[19:]
    df.insert(loc=2, column='observations', value=obs, allow_duplicates=True)

    df.pop('name')

    # solve observations issue

    # nested for loop???
    
    # make time df for each scan


    # make geographic and set crs
    gdf = gpd.GeoDataFrame(df, geometry=gpd.points_from_xy(df.lon, df.lat)) 
    gdf = gdf.set_crs('EPSG:4326')

    #export time gdf into gpkg and csv
    gdf.to_file('scan_data.gpkg', driver="GPKG", layer=i)

    # Check and create save directory for csv files
    savePath = './csvFiles'
    isDir = os.path.isdir(savePath)
    if isDir == False:
        mkdir('csvFiles')
    
    # Save to csv
    gdf.to_csv('csvFiles/'+i+'.csv')

    # DEPRACATED Merge data frames one after another / Mesclar quadros de dados um após o outro
    # DEPRACATED merged = pd.concat([merged,df],ignore_index=True) 
    
# Convert to geodataframe for geographic use / Converter para geodataframe para uso geográfico
# If Z is wanted for the points, add / se Z for desejado para os pontos, adicione ', df.ele' ex. (df.lon, df.lat, df.ele)
# gdf = gpd.GeoDataFrame(merged, geometry=gpd.points_from_xy(merged.lon, merged.lat)) 
# gdf = gdf.set_crs('EPSG:4326')

# gdf

/home/kyle/Nextcloud/Monkey Research/Data_Work/automacao/venvScanAutomation/lib/python3.8/site-packages/geopandas/io/file.py:362: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  pd.Int64Index,
/home/kyle/Nextcloud/Monkey Research/Data_Work/automacao/venvScanAutomation/lib/python3.8/site-packages/geopandas/io/file.py:362: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  pd.Int64Index,


In [3]:
gdf

,date,time,observations,weather,group,observer,lat,lon,ele,geometry
1,2022-06-10,09:44:30,ni3sleep,2,2,,-7.519595,-34.965499,90.0,POINT (-34.96550 -7.51959)
2,2022-06-10,09:45:49,m3,2,2,,-7.519511,-34.965618,90.0,POINT (-34.96562 -7.51951)
3,2022-06-10,09:46:26,m4,2,2,,-7.519585,-34.965594,90.0,POINT (-34.96559 -7.51959)
4,2022-06-10,09:46:40,f4,2,2,,-7.519575,-34.965615,90.0,POINT (-34.96561 -7.51957)
5,2022-06-10,09:47:07,f3,2,2,,-7.519643,-34.965541,90.0,POINT (-34.96554 -7.51964)
...,...,...,...,...,...,...,...,...,...,...
178,2022-06-10,12:42:59,m2,2,2,,-7.520838,-34.965507,79.0,POINT (-34.96551 -7.52084)
179,2022-06-10,12:43:05,ago,2,2,,-7.520940,-34.965517,78.0,POINT (-34.96552 -7.52094)
180,2022-06-10,12:43:21,f3,2,2,,-7.520722,-34.965490,81.0,POINT (-34.96549 -7.52072)
181,2022-06-10,12:43:56,ni3,2,2,,-7.520960,-34.965416,78.0,POINT (-34.96542 -7.52096)


In [ ]:
# # split observations into Age/Sex, Strata position, and behaviour
# ageSex = ''
# strata = ''
# behaviour = ''
# gdf.insert(loc=2, column='age/sex', value=ageSex, allow_duplicates=True)
# gdf.insert(loc=3, column='strata', value=strata, allow_duplicates=True)
# gdf.insert(loc=4, column='behaviour', value=behaviour, allow_duplicates=True)

In [ ]:
# Run loop to identify age, sex, and behaviour
for i, row in gdf.iterrows():
        # asSubstring = gdf['observations'].str[:2]
        if gdf['observations'].str[:2] == 'j2':
                ageSex = 'Juvenile 2'
                strata = gdf['observations'].str[2]
                behaviour = gdf['observations'].str[3:]
                gdf['age/sex']=ageSex
                gdf['strata']=strata
                gdf['behaviour']=behaviour
                

gdf.head()

In [ ]:
gdf.to_csv('fulldata.csv')

In [ ]:
# Attempt to detect each scan / Tente detectar cada varredura

scanStart = gdf.at[0,'time']
scanStart

# scan 40 min interval, apply scan 1. Id any ago's and remove to seperate dataframe or just delete

# remove ago into own scan id



In [ ]:
# export each scan as individual file in gpkg

#### Section 2
Analysis of scans, analyze all of this for every individual scan

In [ ]:
# Find centroid of each scan (collect geometries, find centroid)


In [ ]:
# Distance of each point/animal to centroid


In [ ]:
# points to polygons for area (ha) of group spread (convex hull)


In [ ]:
# Distance between each centroid in temporal order


In [ ]:
# Subgroups/cluster analysis, find clusters on eah scan and distance from each sub-centroid to main group centroid


#### Export Data

In [ ]:
# Export data frame to gpkg for use in QGIS / Exportar quadro de dados para gpkg para uso no QGIS
gdf.to_file('gdf.gpkg', driver="GPKG", layer='gdfExport')
gdf.info()

In [ ]:
gdf.plot()